In [1]:
import sys, os
# assume notebooks/ is your cwd, so src/ is in the parent folder
sys.path.insert(0, os.path.abspath('..'))

# Now your modules are importable
from src.map_viewer_v1 import MapViewer
from src.geocode_v1 import geocode_place
from src.area_calc_v1 import compute_area


# UI widgets
import ipywidgets as widgets
from IPython.display import display, clear_output


# initialize map
mv = MapViewer()
m = mv.get_map()

# output widget for area / messages
out = widgets.Output()


# unit dropdown
unit_selector = widgets.Dropdown(
    options=[('km²','km2'), ('ha','ha'), ('m²','m2')],
    value='km2',
    description='Units:'
)

# place search box + button
search_box = widgets.Text(description='Search:')
search_btn = widgets.Button(description='Go')

def on_search(b):
    with out:
        clear_output()
        try:
            lat, lon = geocode_place(search_box.value)
            m.center = (lat, lon)
            m.zoom = 12
        except Exception as e:
            print(f"Geocoding error: {e}")

search_btn.on_click(on_search)

# layout
controls = widgets.HBox([search_box, search_btn, unit_selector])
display(controls)


def handle_draw(target, action, geo_json):
    # extract the first ring of coordinates
    coords = geo_json['geometry']['coordinates'][0]
    # compute area
    area = compute_area(coords, unit_selector.value)
    # display
    with out:
        clear_output()
        print(f"Area: {area:.3f} {unit_selector.label}")

mv.on_draw(handle_draw)


display(m)
display(out)

Map(center=[20.5937, 78.9629], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

Output()